In [ ]:
import numpy as np
import random
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, chi2
import time


p = 0
while p<1:

    # Define the intervals
    x_interval = [10, 500]
    y_interval = [10, 500]

    # Generate a random x coordinate within the specified interval
    x_coordinate = random.uniform(x_interval[0], x_interval[1])

    # Generate a random y coordinate within the specified interval
    y_coordinate = random.uniform(y_interval[0], y_interval[1])

    # Create a NumPy array representing the sampled point
    sampled_point = np.array([x_coordinate, y_coordinate])

    print("Sampled Point:", sampled_point)

    # Define the interval
    interval = [10, 500]

    # Calculate half of the interval length
    half_interval_length = (interval[1] - interval[0]) / 2

    # Print the result
    print("Half of the interval length:", half_interval_length)



    g = 20 # no. of generations to run (no. of iterations)
    k = 0  # iteration count
    n = 2  # no. of variables in the objective function (dimension)
    l_def = l_ng = l = 4 + math.floor(3*np.log(n))
    l_max = 512*l_def


    F=[]        # To store the function value at each iteration
    Mean = []   # To store the mean value at each iteration
    Cov = []    # To store the covariance matrix at each iteration
    ch = []     # To store the shift in mean at each iteration
    pop = []    # To store the population size at every generation
    sigCSA = [] # To store the step size change by CSA at every generation
    sigTot = [] # To store the total step size change at every generation (step-size correction)
    real_l = [] # To store the found l value
    fixed_l = [] # considered l
    pthetval = []
    psignorm = [] # To consider psignorm values
    hsigma = []
    psigvec = []
    st_sng = []
    st_spg = []
    st_div = []
    adapt = []
    potsig = []
    inval_pg = []
    inval_ng = []
    normval_pg = []
    normval_ng = []
    popchange = []

    l_ng = l_def   # population size starts with default population size
    I = l_ng      # Initial population = I

#     m = np.array([1, 1])
#     c = np.array([[1, 0], [0, 1]])
#     zero = np.array([0,0])
#     psigng = np.array([0,0])
#     pcng = np.array([0,0])     # Because pc = 0 at gen 0
#     p_thetng = np.array([0,0,0,0,0]) # Because p_theta = 0 at gen 0

    m = sampled_point
    c = np.eye(2)
    sig = half_interval_length

    zero = np.zeros(n)
    psigng = np.zeros(n)
    pcng = np.zeros(n)     # Because pc = 0 at gen 0
    p_thetng = np.zeros(int(n*(n+3)/2)) # Because p_theta = 0 at gen 0

    Mean.append(m)
    Cov.append(c)

    real_l.append(l)
    fixed_l.append(l_ng)
    psigvec.append(psigng)

    gam_signg = 0      # normalization factor for p_sigma
    gam_cng = 0        # normalization factor for pc
    gam_thetng = 0     # normalization factor for p_theta
    alpha = 1.4        # for PSA update - population size correction
    beta = 0.4         # for PSA update - p_theta update

    sigTot.append(sig)
    sigCSA.append(sig)

    start_time = time.process_time()

    while k<g:

        I = l_ng
        pc = pcng          # pc equals the pc of next generation
        psig = psigng      # psig equals the psig of next generation
        p_thet = p_thetng  # p_theta equals the p_theta of next generation
        gam_sig = gam_signg
        gam_c = gam_cng
        gam_thet = gam_thetng


        # assigning weights

        mu = math.floor(I/2)
        w_k = []       # weight vector - contains weights for all I
        denom = 0      # denominator for eack w_k_i
        for i in range(1,mu+1):
            denom = denom + (np.log((mu+0.5)) - np.log(i))

        for i in range(1, mu + 1):
            w_k_i = (np.log((mu + 0.5)) - np.log(i)) / denom
            w_k.append(w_k_i)
        for i in range(mu+1,I+1):
            w_k_i = 0
            w_k.append(w_k_i)

        # define mu_eff, c1, c_mu, c_sig, d_sig, cm, Cc

        mu_eff = 1 / np.sum(np.square(w_k))                            # mu_eff = 1/sum(w_i**2)
        c1 = 2/((n+1.3)**2 + mu_eff)                                   # Value from 2018 paper
        c_mu = min(1, (2*(mu_eff - 2 +1)/mu_eff)/((n+2)**2 + mu_eff))  # min(1, (2(mu_eff - 2 +1)/mu_eff)/((n+2)**2 + mu_eff))
        c_sig = (mu_eff+2)/(n+mu_eff+5)
        d_sig = 1 + c_sig  + 2*(max(0, (np.sqrt((mu_eff-1)/(n+1))-1))) # 1 + c_sig + 2*max{ 0, sqrt((mu_eff-1)/(n+1))-1 }
        cm = 1                                                         # Learning parameter is set to 1
        Cc = (4 + (mu_eff/n))/(n + 4 + (2*mu_eff/n))                   # Cc (value from 2018 PSA_CMA_ES paper)

        px=[]
        py=[]

        while(len(px)<I):
            d =  m + sig * np.random.multivariate_normal(zero, c)
            if (d[0] <= 500) and (d[1] <= 500) and (d[0] >= -500) and (d[1] >= -500):
                px.append(d[0])
                py.append(d[1])

        # keep old values
        mpg = m       # old m value
        c_old = c     # old c value
        sigpg = sig   # old sig value


        # select i:l_r values

        b = [(abs(x**2 + y**2 + x*y) + abs(math.sin(x)) + abs(math.cos(y))) for x, y in zip(px, py)]    # Calculate the sum of each px[i]+py[i] and store in list b

        best = sorted(b)     # Sort b in lows

        bx = [px[b.index(val)] for val in best]    # Find the corresponding px[i] and py[i]
        by = [py[b.index(val)] for val in best]    #       values for each element in 'best'

        print("\n")
        print("Candidate solutions")
        for i in range(len(bx)):
            print("At ",k,": (",bx[i],",",by[i],")")

        bestx = []
        besty = []
        for i in range(len(bx)):
            bbx = w_k[i]*(bx[i] - m[0])
            bby = w_k[i]*(by[i] - m[1])
            bestx.append(bbx)
            besty.append(bby)

        dmx = cm * sum(bestx)
        dmy = cm * sum(besty)

        # mean vector update

        mx = m[0] + dmx      # update vector m - x-coordinate
        my = m[1] + dmy      # update vector m - y-coordinate

        m = np.array([mx,my])  # m - next generation

        print("\n")
        print("Mean for ",k+1," = ",m)


        ch.append(m-mpg)   #To calculate the change in mean at each iteration

        print("m[",k+1,"] - m[",k,"]", ch[k])


#         plt.figure(2*k)
#         plt.scatter(px, py, color='b', label='{}th population'.format(k+1))
#         plt.scatter(m[0], m[1], color='r', label='Mean at {}th generation'.format(k+1))
#         plt.xlabel('X')
#         plt.ylabel('Y')
#         plt.title('x[{}] th generation'.format(k + 1))
#         plt.legend()
#         plt.grid()

        # Getting the eigendecompostion of c for the CSA update

        print("\n")
        print("p_sigma path signal understanding")
        eigenvalues, eigenvectors = np.linalg.eig(c) # Eigendecomposition of c
        if any(eigval < 0 for eigval in eigenvalues):
            print("At k=",k+1,", matrix is not PSD")
        B = eigenvectors                             # Ensure eigenvectors are normalized (orthogonal matrix)
        print("Eigenvectors: ")
        print(B)
        D = np.diag(np.sqrt(eigenvalues))            # Diagonal matrix D with square roots of eigenvalues
        print("Eigenvalues")
        print(D)
        D_inv = np.diag(1 / np.sqrt(eigenvalues))    # Calculate the inverse of D
        c_inv_half = B @ D_inv @ B.T                 # Calculate C^(-1/2) = B * D^(-1) * B^T
        print("c_inv_half")
        print(c_inv_half)



        # Evolution path p_sigma for CSA update (p_sigma is 2x1)

        psigng0 = (1-c_sig)*psig[0] + math.sqrt((c_sig*(2 - c_sig)*mu_eff))*(1/sig)*(c_inv_half[0][0]*dmx + c_inv_half[0][1]*dmy)
        psigng1 = (1-c_sig)*psig[1] + math.sqrt((c_sig*(2 - c_sig)*mu_eff))*(1/sig)*(c_inv_half[1][0]*dmx + c_inv_half[1][1]*dmy)
        gam_signg = ((1-c_sig)**2)*gam_sig + c_sig*(2-c_sig)   # normalization p_sigma

        psigng = np.array([psigng0,psigng1])

        psigvec.append(psigng)

        print("\n")
        print("Evolution path p_sigma = ",psigng)

        # Evolution path pc for rank-one update (pc is 2x1)

        chi = (math.sqrt(n))*(1 - (1/(4*n)) + (1/(21*n**2))) # E[||N(0,I)||]
        psig_vector = np.array([[psigng[0]], [psigng[1]]])   # Define a psigng as a 2x1 matrix
        norm_psig = np.linalg.norm(psig_vector)              # Calculate the 2-norm (Euclidean norm)

        if norm_psig < chi:
            print("norm_psig < chi --> CSA decreased")
        else:
            print("norm_psig >= chi --> CSA increased")
        cond = (1.4 + 2/(n+1))*chi*(math.sqrt(gam_signg))    # condition for h_sigma

        psignorm.append(norm_psig)

        if norm_psig<cond:
            h_sig = 1
        else:
            h_sig = 0

        hsigma.append(h_sig)

        pcng0 = (1-Cc)*pc[0] + h_sig*(math.sqrt((Cc*(2 - Cc)*mu_eff)))*(1/sig)*dmx  # x-coordinate
        pcng1 = (1-Cc)*pc[1] + h_sig*(math.sqrt((Cc*(2 - Cc)*mu_eff)))*(1/sig)*dmy  # y-coordinate
        gam_cng = ((1-Cc)**2)*gam_c + h_sig*Cc*(2-Cc)   # normalization pc

        pcng = np.array([pcng0,pcng1])

        sigA = sig

        # step size update (sigma)

        exp_val = (c_sig/d_sig)*((norm_psig/chi) - (math.sqrt(gam_signg)))
        sig = sig*(math.exp(exp_val))

        sigB = sig

        print("||P_sig||/chi = ",(norm_psig/chi))
        print("\n")
        print("sig * exp(",exp_val,")")
        print("step-size after CSA correction : ")
        print(sig)

        sigCSA.append(sig)



        # rank-mu update

        dift_x = []  # string to hold x - m := x-coordinate
        dift_y = []  # string to hold x - m := y-coordinate
        cmat00 = []  # 00 element of (x-m).(x-m)^T outer product
        cmat01 = []  # 01 element of (x-m).(x-m)^T outer product
        cmat10 = []  # 10 element of (x-m).(x-m)^T outer product
        cmat11 = []  # 11 element of (x-m).(x-m)^T outer product

        for i in range(len(bx)):
            difx = bx[i] - m[0]
            dify = by[i] - m[1]
            dift_x.append(difx)
            dift_y.append(dify)
        for i in range(len(bx)):
            c00 = w_k[i]*(dift_x[i]*dift_x[i] - c[0][0])
            c01 = w_k[i]*(dift_x[i]*dift_y[i] - c[0][1])
            c10 = w_k[i]*(dift_x[i]*dift_y[i] - c[1][0])
            c11 = w_k[i]*(dift_y[i]*dift_y[i] - c[1][1])
            cmat00.append(c00)
            cmat01.append(c01)
            cmat10.append(c10)
            cmat11.append(c11)

        cRank_mu = np.zeros((2,2))
        cRank_mu[0][0] = sum(cmat00)
        cRank_mu[0][1] = sum(cmat01)
        cRank_mu[1][0] = sum(cmat10)
        cRank_mu[1][1] = sum(cmat11)


        # rank-one update

        cRank_1 = np.zeros((2,2))
        cRank_1[0][0] = pcng[0]*pcng[0] - gam_cng*c[0][0]
        cRank_1[0][1] = pcng[0]*pcng[1] - gam_cng*c[0][1]
        cRank_1[1][0] = pcng[1]*pcng[0] - gam_cng*c[1][0]
        cRank_1[1][1] = pcng[1]*pcng[1] - gam_cng*c[1][1]

        # Covariance matrix update

        c = c + c1*cRank_1 + c_mu*cRank_mu



#         ##    ***  PSA update  ***

#         # 1 - Approximate FIM

#         fim = c_inv_half

#         # 2 - Calculate Deltha_Theta

#         dtheta = np.array([0,0,0,0,0])
#         dtheta0 = fim[0][0]*(dmx) + fim[0][1]*(dmy)
#         dtheta1 = fim[0][1]*(dmx) + fim[1][1]*(dmy)
#         dtheta2 = (sig**2)*(c[0][0]) - (sigpg**2)*(c_old[0][0])
#         dtheta3 = (sig**2)*(c[0][1]) - (sigpg**2)*(c_old[0][1])
#         dtheta4 = (sig**2)*(c[1][1]) - (sigpg**2)*(c_old[1][1])

#         print("\n")
#         print("I^(1/2)*Dtheta (5x1)-vector")


#         dtheta = np.array([dtheta0,dtheta1,dtheta2,dtheta3,dtheta4])

#         print("     dmx         dmy    sig*dC[0][0] sig*dC[0][1] sig*dC[1][1]")
#         print(dtheta)

#         # 3 - Find terms of E[ ||.|| ]

#         w_k_i3 = 0                        # sum(wi^3)
#         for i in range(len(w_k)):
#             w_k_i3 = w_k_i3 + w_k[i]**3

#         E1  = n*(cm**2)/mu_eff
#         E2  = (2*n)*((n - chi**2)/(chi**2))*gam_signg*((c_sig/d_sig)**2)
#         E3a = (n**2 + n)*(c_mu**2)/mu_eff
#         E3b = (n**2 + n)*(Cc)*(2 - Cc)*(c1)*(c_mu)*(mu_eff)*(w_k_i3)
#         E3c = (c1**2)*(gam_cng**2)*(n**2)
#         E3d = 1 - (2*gam_cng) + 2*n*(gam_cng**2)
#         E3e = 1 + (8*(gam_signg)*((n - chi**2)/(chi**2))*((c_sig/d_sig)**2))
#         E3  = (1/2)*E3e*(E3a + E3b + E3c + E3d)

#         # 4 - Approximation for E[ ||.|| ]

#         Efim = E1 + E2 + E3

#         Efim_rt = math.sqrt(Efim)  # E[|| fim.dtheta ||]^(1/2)

#         # Evolution path p_theta for PSA update (p_theta is 5x1) ( R^(n(n+3)/2) )

#         p_thetng0 = (1 - beta)*p_thet[0] + (1/Efim_rt)*(math.sqrt((beta*(2 - beta))))*(dtheta[0])
#         p_thetng1 = (1 - beta)*p_thet[1] + (1/Efim_rt)*(math.sqrt((beta*(2 - beta))))*(dtheta[1])
#         p_thetng2 = (1 - beta)*p_thet[2] + (1/Efim_rt)*(math.sqrt((beta*(2 - beta))))*(dtheta[2])
#         p_thetng3 = (1 - beta)*p_thet[3] + (1/Efim_rt)*(math.sqrt((beta*(2 - beta))))*(dtheta[3])
#         p_thetng4 = (1 - beta)*p_thet[4] + (1/Efim_rt)*(math.sqrt((beta*(2 - beta))))*(dtheta[4])

#         gam_thetng = ((1 - beta)**2)*gam_thet + beta*(2 - beta)
#         p_thetng = np.array([[p_thetng0], [p_thetng1], [p_thetng2], [p_thetng3], [p_thetng4]])

#         print("p_theta evolution path (5x1)")
#         print(p_thetng)

#         # Update population size

#         norm_pthet = np.linalg.norm(p_thetng)     # Calculate the 2-norm (Euclidean norm) of p_theta

#         exp_val2 = beta*(gam_thetng - (norm_pthet**2)/alpha)

#         print("beta*exp(val2)")
#         print("val2 = ",(gam_thetng - (norm_pthet**2)/alpha))

#         print("norm of ptheta = ",norm_pthet)
#         print("\n")

#         pthetval.append(norm_pthet)

#         l = l*math.exp(exp_val2)

#         print("real l = ",l)

#         l = min(max(l,l_def),l_max)
#         l_pg = l_ng   # keep old population size
#         l_ng = round(l)

#         fixed_l.append(l_ng)
#         print("fixd l = ",l_ng)


        ## change up pop-size to understand significant level

        l_pg = l_ng   # keep old population size
        print("\n")
        print("Add",k,"to prev pop size (",l_pg,"): ")
        l_ng = l_pg + k
        print("pop size for g^(",k+1,") = ",k,"+",l_pg," = ",l_ng)
        real_l.append(l_ng)




        print("\n")
        print("Step size correction")

        # step size correction

        # Generating (lamda_r)^(t+1) to find E[N_i:Lr]

        lpx = []
        lpy = []

        while(len(lpx)<l_ng):
            ld =  m + sig * np.random.multivariate_normal(zero, c)
            if (ld[0] <= 500) and (ld[1] <= 500) and (ld[0] >= -500) and (ld[1] >= -500):
                lpx.append(ld[0])
                lpy.append(ld[1])

        lb = [(abs(x**2 + y**2 + x*y) + abs(math.sin(x)) + abs(math.cos(y))) for x, y in zip(lpx, lpy)]    # Calculate the sum of each px[i]+py[i] and store in list b

        lbest = sorted(lb)     # Sort b in lows

        lbx = [lpx[lb.index(val)] for val in lbest]    # Find the corresponding px[i] and py[i]
        lby = [lpy[lb.index(val)] for val in lbest]    #       values for each element in 'best'

        # weights for (lamda_r)^(t+1)
        lw_k = []       # weight vector - contains weights for all I
        ldenom = 0      # denominator for eack w_k_i
        lmu = math.floor(l_ng/2)
        for i in range(1,l_ng+1):
            ldenom = ldenom + (np.log((l_ng+0.5)) - np.log(i))

        for i in range(1, lmu + 1):
            lw_k_i = (np.log((l_ng + 0.5)) - np.log(i)) / ldenom
            lw_k.append(lw_k_i)
        for i in range(lmu+1,l_ng+1):
            lw_k_i = 0
            lw_k.append(lw_k_i)

        lmu_eff = 1 / np.sum(np.square(lw_k))

        #finding phi^(-1)
        al = 0.375
        prob_pg = []
        prob_ng = []

        inval_pg = []
        normval_pg = []

        for i in range(len(bx)):
            valp = (i+1-al)/(len(bx)+1-2*al)
            inval_pg.append(valp)
            phipg = norm.ppf(valp)
            normval_pg.append(phipg)

            mxppg = w_k[i]*(mpg[0] + sigpg*(phipg))
            myppg = w_k[i]*(mpg[1] + sigpg*(phipg))
            exp_pg = math.sqrt((mxppg**2 + myppg**2))
            prob_pg.append(exp_pg)


#             mxppg = w_k[i]*(mpg[0] + sigpg*(norm.ppf((i+1)/(len(bx)+1))))
#             myppg = w_k[i]*(mpg[1] + sigpg*(norm.ppf((i+1)/(len(bx)+1))))
#             exp_pg = math.sqrt((mxppg**2 + myppg**2))
#             prob_pg.append(exp_pg)
            #exp_pg = mxppg

        print("\n")
        print("At generation",k,"P(x) value for L_g =")
        print(inval_pg)
        print("At generation",k,"phi(-1) value for L_g =")
        print(normval_pg)
        print("\n")


        inval_ng = []
        normval_ng = []

        for i in range(len(lbx)):
            valn = (i+1-al)/(len(lbx)+1-2*al)
            inval_ng.append(valn)
            phing = norm.ppf(valn)
            normval_ng.append(phing)

            mxpng = lw_k[i]*(m[0] + sig*(phing))
            mypng = lw_k[i]*(m[1] + sig*(phing))
            exp_ng = math.sqrt((mxpng**2 + mypng**2))
            prob_ng.append(exp_ng)

#             mxpng = lw_k[i]*(m[0] + sig*(norm.ppf((i+1)/(len(lbx)+1))))
#             mypng = lw_k[i]*(m[1] + sig*(norm.ppf((i+1)/(len(lbx)+1))))
#             exp_ng = math.sqrt((mxpng**2 + mypng**2))
#             #exp_ng = mxpng
#             prob_ng.append(exp_ng)
        print("\n")
        print("At generation",k,"P(x) value for L_g+1 =")
        print(inval_ng)
        print("At generation",k,"phi(-1) value for L_g+1 =")
        print(normval_ng)
        print("\n")


        print("phi^(-1)_pg = ",phipg)
        print("phi^(-1)_ng = ",phing)

        print("\n")

        print("weighted expected values of NOS")
        print("\n")

        print("E[l_r^(g)] = ",prob_pg)
        print("E[l_r^(g+1)] = ",prob_ng)

        print("sum{E[l_r^(g)]} = ",sum(prob_pg))
        print("sum{E[l_r^(g+1)]} = ",sum(prob_ng))

        print("\n")

        cpg = (-1)*sum(prob_pg)
        cng = (-1)*sum(prob_ng)
        sigfunc_pg = (cpg*n*mu_eff)/(n - 1 + (cpg**2)*mu_eff)
        sigfunc_ng = (cng*n*lmu_eff)/(n - 1 + (cng**2)*lmu_eff)

        print("lr(g+1) = ",sigfunc_ng)
        print("lr(g) = ",sigfunc_pg)
        print("lr(g+1)/lr(g) = ",sigfunc_ng/sigfunc_pg)
        print("\n")
        print("st-s b4 correction = ",sig)
        # step size correction

        st_sng.append(sigfunc_ng)
        st_spg.append(sigfunc_pg)
        st_div.append(sigfunc_ng/sigfunc_pg)

        popchange.append(abs(l_ng - l_pg))


#         if reall - l < 10:
#             if norm_psig >= chi:
#                 sig = sig
#                 print("step-size correction not done as CSA request step-size increase")
#             else:
#                 sig = sig*(0.5*(sigfunc_ng/sigfunc_pg))
#                 print("scaled step-size correction (pop-dif < 10)")

#         else:
#             if norm_psig >= chi:
#                 sig = sig
#                 print("step-size correction not done as CSA request step-size increase")
#             else:
#                 sig = sig*((sigfunc_ng/sigfunc_pg))
#                 print("unscaled step-size correction")


#         print("\n")
#         print("Trying to figure out why this formulation worked")
#         print("reall = ",reall)
#         print("l = ",l)
#         print("If ",reall,"-",l,"< 10 --> sigma was scaled (if psig okay)")
#         print("\n")

        sig = sig*((sigfunc_ng/sigfunc_pg))





#         if (sum(sum(D)))<10**(-2):
#             sig = sig*(0.5*(sigfunc_ng/sigfunc_pg))
#         else:
#             sig = sig*((sigfunc_ng/sigfunc_pg))



#         sig = sig

        print("Step size correction = sig*[lr(g+1)/lr(g)]")
        print("st-x af correction = ",sig)

        sigTot.append(sig)



        f = (abs(mx**2 + my**2 + mx*my) + abs(math.sin(mx)) + abs(math.cos(my)))   # function value - to plot

        F.append(f)
        Mean.append(m)
        Cov.append(c)

        print("\n")
        print("Covariance matrix for ",k+1)
        print(c)


#         # Create a grid of x and y values
        x = np.linspace(-100, 100, 500)
        y = np.linspace(-100, 100, 500)
        X, Y = np.meshgrid(x, y)

        Z = np.exp(-(np.square(X - m[0]) * c[0, 0] +
                 (X - m[0]) * (Y - m[1]) * c[0, 1] +
                 np.square(Y - m[1]) * c[1, 1])) / (2 * np.pi * np.sqrt(np.linalg.det(c)))

        # Plot the 2D Gaussian distribution
        plt.figure(3*k+1)
        plt.figure(figsize=(8, 6))
        plt.contourf(X, Y, Z, levels=10, cmap='Reds')
        plt.colorbar()
        plt.scatter(px, py, color='cyan', alpha=0.4, label='xk({})'.format(k+1))
        plt.scatter(m[0], m[1], color='yellow', label='Mean at {}th generation'.format(k+1))
        plt.xlabel('X',fontsize=14)
        plt.ylabel('Y',fontsize=14)
        plt.tick_params(axis='both', which='major', labelsize=16)
        plt.title('2D Gaussian Distribution for generation {}'.format(k+1),fontsize=16)
        plt.grid(True)
        plt.show()



#         # Generate random points from multivariate normal distribution
#         num_points = len(px)
#         points = np.random.multivariate_normal(m, c, num_points)
#         px, py = points[:, 0], points[:, 1]

#         # Create a multivariate normal distribution object
#         mv_normal = multivariate_normal(mean=m, cov=c)

#         # Define confidence level
#         confidence_level = 0.95

#         # Degrees of freedom for the chi-squared distribution
#         df = len(m)

#         # Calculate critical value for the confidence interval
#         critical_value = chi2.ppf(confidence_level, df)

#         # Plot points
#         plt.scatter(px, py, color='blue', label='Generated Points')

#         # Plot contours
#         x, y = np.mgrid[-100:100:.01, -100:100:.01]
#         pos = np.dstack((x, y))
#         contour_levels = mv_normal.pdf(pos)
#         plt.contour(x, y, contour_levels, colors='blue',alpha=0.3)

#         # Plot critical value contour
#         plt.contour(x, y, mv_normal.pdf(pos), levels=[mv_normal.pdf(critical_value)], colors='green')

#         # Identify points inside and outside the confidence interval
#         inside_points = points[np.where(mv_normal.pdf(points) >= mv_normal.pdf(critical_value))]
#         outside_points = points[np.where(mv_normal.pdf(points) < mv_normal.pdf(critical_value))]

#         # Highlight points
#         plt.scatter(inside_points[:, 0], inside_points[:, 1], color='green', label='Inside Confidence Interval')
#         plt.scatter(outside_points[:, 0], outside_points[:, 1], color='red', label='Outside Confidence Interval')

#         # Add legend
#         plt.legend()

#         # Show plot
#         plt.xlabel('X')
#         plt.ylabel('Y')
#         plt.title('Points from Multivariate Normal Distribution with Confidence Intervals')
#         plt.grid(True)
#         plt.show()




        print("\n")
        print("\n")
        print("\n")
        print("\n")

        if abs(F[k] - 1.0)<10**(-2):
            break

        k = k+1



    end_time = time.process_time()

    print("\n")

    cpu_time = end_time - start_time
    print("CPU time for the loop:", cpu_time)

    print("run ",p," : fun_vals = ",F)
    p = p+1


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 12))

# Define a color map for shading points from light to dark
colors = plt.cm.Blues_r([i / len(rpopchange) for i in range(len(rpopchange))])

# Dictionary to store positions already annotated
annotated_positions = {}

# Plot each point with shading and annotate with generation number
for i, (x, y) in enumerate(zip(rpopchange, adsigCSA)):
    plt.scatter(x, y, color=colors[i], label=f'Generation {i+1}')
    # Adjust text position if position already annotated
    if (x, y) in annotated_positions:
        dx = dy = 50  # Adjust this value as needed
        x += dx
        y += dy
    plt.text(x, y, str(i+1), fontsize=13, ha='right', va='bottom')
    annotated_positions[(x, y)] = True

plt.title('Real Population Change vs Step Size',fontsize=16)
plt.xlabel('Population size change = N^(g+1) - N^(g)',fontsize=16)
plt.ylabel('Step Size = sigma',fontsize = 16)
plt.grid(True)
plt.legend()
plt.show()



plt.figure(figsize=(15, 12))

# Define a color map for shading points from light to dark
colors = plt.cm.Blues_r([i / len(real_l) for i in range(len(real_l))])

# Dictionary to store positions already annotated
annotated_positions = {}

# Plot each point with shading and annotate with generation number
for i, (x, y) in enumerate(zip(real_l, sigCSA)):
    plt.scatter(x, y, color=colors[i], label=f'Generation {i+1}')
    # Adjust text position if position already annotated
    if (x, y) in annotated_positions:
        dx = dy = 50  # Adjust this value as needed
        x += dx
        y += dy
    plt.text(x, y, str(i), fontsize=13, ha='right', va='bottom')
    annotated_positions[(x, y)] = True

plt.title('Population size vs Step Size',fontsize=16)
plt.xlabel('Population size = N^(g)',fontsize=16)
plt.ylabel('Step Size = sigma',fontsize = 16)
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
import numpy as np
import random
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, chi2
import time


p = 0
while p<1:

    # Define the intervals
    x_interval = [10, 100]
    y_interval = [10, 100]

    # Generate a random x coordinate within the specified interval
    x_coordinate = random.uniform(x_interval[0], x_interval[1])

    # Generate a random y coordinate within the specified interval
    y_coordinate = random.uniform(y_interval[0], y_interval[1])

    # Create a NumPy array representing the sampled point
    sampled_point = np.array([x_coordinate, y_coordinate])

    print("Sampled Point:", sampled_point)

    # Define the interval
    interval = [10, 100]

    # Calculate half of the interval length
    half_interval_length = (interval[1] - interval[0]) / 2

    # Print the result
    print("Half of the interval length:", half_interval_length)



    g = 20 # no. of generations to run (no. of iterations)
    k = 0  # iteration count
    n = 2  # no. of variables in the objective function (dimension)
    l_def = l_ng = l = 4 + math.floor(3*np.log(n))
    l_max = 512*l_def


    F=[]        # To store the function value at each iteration
    Mean = []   # To store the mean value at each iteration
    Cov = []    # To store the covariance matrix at each iteration
    ch = []     # To store the shift in mean at each iteration
    pop = []    # To store the population size at every generation
    sigCSA = [] # To store the step size change by CSA at every generation
    sigTot = [] # To store the total step size change at every generation (step-size correction)
    real_l = [] # To store the found l value
    fixed_l = [] # considered l
    pthetval = []
    psignorm = [] # To consider psignorm values
    hsigma = []
    psigvec = []
    st_sng = []
    st_spg = []
    st_div = []
    adapt = []
    potsig = []
    inval_pg = []
    inval_ng = []
    normval_pg = []
    normval_ng = []
    popchange = []

    l_ng = l_def   # population size starts with default population size
    I = l_ng      # Initial population = I

#     m = np.array([1, 1])
#     c = np.array([[1, 0], [0, 1]])
#     zero = np.array([0,0])
#     psigng = np.array([0,0])
#     pcng = np.array([0,0])     # Because pc = 0 at gen 0
#     p_thetng = np.array([0,0,0,0,0]) # Because p_theta = 0 at gen 0

    m = sampled_point
    c = np.eye(2)
    sig = half_interval_length

    zero = np.zeros(n)
    psigng = np.zeros(n)
    pcng = np.zeros(n)     # Because pc = 0 at gen 0
    p_thetng = np.zeros(int(n*(n+3)/2)) # Because p_theta = 0 at gen 0

    Mean.append(m)
    Cov.append(c)

    real_l.append(l)
    fixed_l.append(l_ng)
    psigvec.append(psigng)

    gam_signg = 0      # normalization factor for p_sigma
    gam_cng = 0        # normalization factor for pc
    gam_thetng = 0     # normalization factor for p_theta
    alpha = 1.4        # for PSA update - population size correction
    beta = 0.4         # for PSA update - p_theta update

    sigTot.append(sig)
    sigCSA.append(sig)

    start_time = time.process_time()

    while k<g:

        I = l_ng
        pc = pcng          # pc equals the pc of next generation
        psig = psigng      # psig equals the psig of next generation
        p_thet = p_thetng  # p_theta equals the p_theta of next generation
        gam_sig = gam_signg
        gam_c = gam_cng
        gam_thet = gam_thetng


        # assigning weights

        mu = math.floor(I/2)
        w_k = []       # weight vector - contains weights for all I
        denom = 0      # denominator for eack w_k_i
        for i in range(1,mu+1):
            denom = denom + (np.log((mu+0.5)) - np.log(i))

        for i in range(1, mu + 1):
            w_k_i = (np.log((mu + 0.5)) - np.log(i)) / denom
            w_k.append(w_k_i)
        for i in range(mu+1,I+1):
            w_k_i = 0
            w_k.append(w_k_i)

        # define mu_eff, c1, c_mu, c_sig, d_sig, cm, Cc

        mu_eff = 1 / np.sum(np.square(w_k))                            # mu_eff = 1/sum(w_i**2)
        c1 = 2/((n+1.3)**2 + mu_eff)                                   # Value from 2018 paper
        c_mu = min(1, (2*(mu_eff - 2 +1)/mu_eff)/((n+2)**2 + mu_eff))  # min(1, (2(mu_eff - 2 +1)/mu_eff)/((n+2)**2 + mu_eff))
        c_sig = (mu_eff+2)/(n+mu_eff+5)
        d_sig = 1 + c_sig  + 2*(max(0, (np.sqrt((mu_eff-1)/(n+1))-1))) # 1 + c_sig + 2*max{ 0, sqrt((mu_eff-1)/(n+1))-1 }
        cm = 1                                                         # Learning parameter is set to 1
        Cc = (4 + (mu_eff/n))/(n + 4 + (2*mu_eff/n))                   # Cc (value from 2018 PSA_CMA_ES paper)

        px=[]
        py=[]

        while(len(px)<I):
            d =  m + sig * np.random.multivariate_normal(zero, c)
            if (d[0] <= 10) and (d[1] <= 10) and (d[0] >= -10) and (d[1] >= -10):
                px.append(d[0])
                py.append(d[1])

        # keep old values
        mpg = m       # old m value
        c_old = c     # old c value
        sigpg = sig   # old sig value


        # select i:l_r values

        b = [(-0.0001 * ((np.abs(np.sin(x) * np.sin(y) * np.exp(np.abs(100 - np.sqrt(x**2 + y**2) / np.pi)))) + 1)**0.1) for x, y in zip(px, py)]    # Calculate the sum of each px[i]+py[i] and store in list b

        best = sorted(b)     # Sort b in lows

        bx = [px[b.index(val)] for val in best]    # Find the corresponding px[i] and py[i]
        by = [py[b.index(val)] for val in best]    #       values for each element in 'best'

        print("\n")
        print("Candidate solutions")
        for i in range(len(bx)):
            print("At ",k,": (",bx[i],",",by[i],")")

        bestx = []
        besty = []
        for i in range(len(bx)):
            bbx = w_k[i]*(bx[i] - m[0])
            bby = w_k[i]*(by[i] - m[1])
            bestx.append(bbx)
            besty.append(bby)

        dmx = cm * sum(bestx)
        dmy = cm * sum(besty)

        # mean vector update

        mx = m[0] + dmx      # update vector m - x-coordinate
        my = m[1] + dmy      # update vector m - y-coordinate

        m = np.array([mx,my])  # m - next generation

        print("\n")
        print("Mean for ",k+1," = ",m)


        ch.append(m-mpg)   #To calculate the change in mean at each iteration

        print("m[",k+1,"] - m[",k,"]", ch[k])


#         plt.figure(2*k)
#         plt.scatter(px, py, color='b', label='{}th population'.format(k+1))
#         plt.scatter(m[0], m[1], color='r', label='Mean at {}th generation'.format(k+1))
#         plt.xlabel('X')
#         plt.ylabel('Y')
#         plt.title('x[{}] th generation'.format(k + 1))
#         plt.legend()
#         plt.grid()

        # Getting the eigendecompostion of c for the CSA update

        print("\n")
        print("p_sigma path signal understanding")
        eigenvalues, eigenvectors = np.linalg.eig(c) # Eigendecomposition of c
        if any(eigval < 0 for eigval in eigenvalues):
            print("At k=",k+1,", matrix is not PSD")
        B = eigenvectors                             # Ensure eigenvectors are normalized (orthogonal matrix)
        print("Eigenvectors: ")
        print(B)
        D = np.diag(np.sqrt(eigenvalues))            # Diagonal matrix D with square roots of eigenvalues
        print("Eigenvalues")
        print(D)
        D_inv = np.diag(1 / np.sqrt(eigenvalues))    # Calculate the inverse of D
        c_inv_half = B @ D_inv @ B.T                 # Calculate C^(-1/2) = B * D^(-1) * B^T
        print("c_inv_half")
        print(c_inv_half)



        # Evolution path p_sigma for CSA update (p_sigma is 2x1)

        psigng0 = (1-c_sig)*psig[0] + math.sqrt((c_sig*(2 - c_sig)*mu_eff))*(1/sig)*(c_inv_half[0][0]*dmx + c_inv_half[0][1]*dmy)
        psigng1 = (1-c_sig)*psig[1] + math.sqrt((c_sig*(2 - c_sig)*mu_eff))*(1/sig)*(c_inv_half[1][0]*dmx + c_inv_half[1][1]*dmy)
        gam_signg = ((1-c_sig)**2)*gam_sig + c_sig*(2-c_sig)   # normalization p_sigma

        psigng = np.array([psigng0,psigng1])

        psigvec.append(psigng)

        print("\n")
        print("Evolution path p_sigma = ",psigng)

        # Evolution path pc for rank-one update (pc is 2x1)

        chi = (math.sqrt(n))*(1 - (1/(4*n)) + (1/(21*n**2))) # E[||N(0,I)||]
        psig_vector = np.array([[psigng[0]], [psigng[1]]])   # Define a psigng as a 2x1 matrix
        norm_psig = np.linalg.norm(psig_vector)              # Calculate the 2-norm (Euclidean norm)

        if norm_psig < chi:
            print("norm_psig < chi --> CSA decreased")
        else:
            print("norm_psig >= chi --> CSA increased")
        cond = (1.4 + 2/(n+1))*chi*(math.sqrt(gam_signg))    # condition for h_sigma

        psignorm.append(norm_psig)

        if norm_psig<cond:
            h_sig = 1
        else:
            h_sig = 0

        hsigma.append(h_sig)

        pcng0 = (1-Cc)*pc[0] + h_sig*(math.sqrt((Cc*(2 - Cc)*mu_eff)))*(1/sig)*dmx  # x-coordinate
        pcng1 = (1-Cc)*pc[1] + h_sig*(math.sqrt((Cc*(2 - Cc)*mu_eff)))*(1/sig)*dmy  # y-coordinate
        gam_cng = ((1-Cc)**2)*gam_c + h_sig*Cc*(2-Cc)   # normalization pc

        pcng = np.array([pcng0,pcng1])

        sigA = sig

        # step size update (sigma)

        exp_val = (c_sig/d_sig)*((norm_psig/chi) - (math.sqrt(gam_signg)))
        sig = sig*(math.exp(exp_val))

        sigB = sig

        print("||P_sig||/chi = ",(norm_psig/chi))
        print("\n")
        print("sig * exp(",exp_val,")")
        print("step-size after CSA correction : ")
        print(sig)

        sigCSA.append(sig)



        # rank-mu update

        dift_x = []  # string to hold x - m := x-coordinate
        dift_y = []  # string to hold x - m := y-coordinate
        cmat00 = []  # 00 element of (x-m).(x-m)^T outer product
        cmat01 = []  # 01 element of (x-m).(x-m)^T outer product
        cmat10 = []  # 10 element of (x-m).(x-m)^T outer product
        cmat11 = []  # 11 element of (x-m).(x-m)^T outer product

        for i in range(len(bx)):
            difx = bx[i] - m[0]
            dify = by[i] - m[1]
            dift_x.append(difx)
            dift_y.append(dify)
        for i in range(len(bx)):
            c00 = w_k[i]*(dift_x[i]*dift_x[i] - c[0][0])
            c01 = w_k[i]*(dift_x[i]*dift_y[i] - c[0][1])
            c10 = w_k[i]*(dift_x[i]*dift_y[i] - c[1][0])
            c11 = w_k[i]*(dift_y[i]*dift_y[i] - c[1][1])
            cmat00.append(c00)
            cmat01.append(c01)
            cmat10.append(c10)
            cmat11.append(c11)

        cRank_mu = np.zeros((2,2))
        cRank_mu[0][0] = sum(cmat00)
        cRank_mu[0][1] = sum(cmat01)
        cRank_mu[1][0] = sum(cmat10)
        cRank_mu[1][1] = sum(cmat11)


        # rank-one update

        cRank_1 = np.zeros((2,2))
        cRank_1[0][0] = pcng[0]*pcng[0] - gam_cng*c[0][0]
        cRank_1[0][1] = pcng[0]*pcng[1] - gam_cng*c[0][1]
        cRank_1[1][0] = pcng[1]*pcng[0] - gam_cng*c[1][0]
        cRank_1[1][1] = pcng[1]*pcng[1] - gam_cng*c[1][1]

        # Covariance matrix update

        c = c + c1*cRank_1 + c_mu*cRank_mu



#         ##    ***  PSA update  ***

#         # 1 - Approximate FIM

#         fim = c_inv_half

#         # 2 - Calculate Deltha_Theta

#         dtheta = np.array([0,0,0,0,0])
#         dtheta0 = fim[0][0]*(dmx) + fim[0][1]*(dmy)
#         dtheta1 = fim[0][1]*(dmx) + fim[1][1]*(dmy)
#         dtheta2 = (sig**2)*(c[0][0]) - (sigpg**2)*(c_old[0][0])
#         dtheta3 = (sig**2)*(c[0][1]) - (sigpg**2)*(c_old[0][1])
#         dtheta4 = (sig**2)*(c[1][1]) - (sigpg**2)*(c_old[1][1])

#         print("\n")
#         print("I^(1/2)*Dtheta (5x1)-vector")


#         dtheta = np.array([dtheta0,dtheta1,dtheta2,dtheta3,dtheta4])

#         print("     dmx         dmy    sig*dC[0][0] sig*dC[0][1] sig*dC[1][1]")
#         print(dtheta)

#         # 3 - Find terms of E[ ||.|| ]

#         w_k_i3 = 0                        # sum(wi^3)
#         for i in range(len(w_k)):
#             w_k_i3 = w_k_i3 + w_k[i]**3

#         E1  = n*(cm**2)/mu_eff
#         E2  = (2*n)*((n - chi**2)/(chi**2))*gam_signg*((c_sig/d_sig)**2)
#         E3a = (n**2 + n)*(c_mu**2)/mu_eff
#         E3b = (n**2 + n)*(Cc)*(2 - Cc)*(c1)*(c_mu)*(mu_eff)*(w_k_i3)
#         E3c = (c1**2)*(gam_cng**2)*(n**2)
#         E3d = 1 - (2*gam_cng) + 2*n*(gam_cng**2)
#         E3e = 1 + (8*(gam_signg)*((n - chi**2)/(chi**2))*((c_sig/d_sig)**2))
#         E3  = (1/2)*E3e*(E3a + E3b + E3c + E3d)

#         # 4 - Approximation for E[ ||.|| ]

#         Efim = E1 + E2 + E3

#         Efim_rt = math.sqrt(Efim)  # E[|| fim.dtheta ||]^(1/2)

#         # Evolution path p_theta for PSA update (p_theta is 5x1) ( R^(n(n+3)/2) )

#         p_thetng0 = (1 - beta)*p_thet[0] + (1/Efim_rt)*(math.sqrt((beta*(2 - beta))))*(dtheta[0])
#         p_thetng1 = (1 - beta)*p_thet[1] + (1/Efim_rt)*(math.sqrt((beta*(2 - beta))))*(dtheta[1])
#         p_thetng2 = (1 - beta)*p_thet[2] + (1/Efim_rt)*(math.sqrt((beta*(2 - beta))))*(dtheta[2])
#         p_thetng3 = (1 - beta)*p_thet[3] + (1/Efim_rt)*(math.sqrt((beta*(2 - beta))))*(dtheta[3])
#         p_thetng4 = (1 - beta)*p_thet[4] + (1/Efim_rt)*(math.sqrt((beta*(2 - beta))))*(dtheta[4])

#         gam_thetng = ((1 - beta)**2)*gam_thet + beta*(2 - beta)
#         p_thetng = np.array([[p_thetng0], [p_thetng1], [p_thetng2], [p_thetng3], [p_thetng4]])

#         print("p_theta evolution path (5x1)")
#         print(p_thetng)

#         # Update population size

#         norm_pthet = np.linalg.norm(p_thetng)     # Calculate the 2-norm (Euclidean norm) of p_theta

#         exp_val2 = beta*(gam_thetng - (norm_pthet**2)/alpha)

#         print("beta*exp(val2)")
#         print("val2 = ",(gam_thetng - (norm_pthet**2)/alpha))

#         print("norm of ptheta = ",norm_pthet)
#         print("\n")

#         pthetval.append(norm_pthet)

#         l = l*math.exp(exp_val2)

#         print("real l = ",l)

#         l = min(max(l,l_def),l_max)
#         l_pg = l_ng   # keep old population size
#         l_ng = round(l)

#         fixed_l.append(l_ng)
#         print("fixd l = ",l_ng)


        ## change up pop-size to understand significant level

        l_pg = l_ng   # keep old population size
        print("\n")
        print("Add",k,"to prev pop size (",l_pg,"): ")
        l_ng = l_pg + k
        print("pop size for g^(",k+1,") = ",k,"+",l_pg," = ",l_ng)
        real_l.append(l_ng)




        print("\n")
        print("Step size correction")

        # step size correction

        # Generating (lamda_r)^(t+1) to find E[N_i:Lr]

        lpx = []
        lpy = []

        while(len(lpx)<l_ng):
            ld =  m + sig * np.random.multivariate_normal(zero, c)
            if (ld[0] <= 10) and (ld[1] <= 10) and (ld[0] >= -10) and (ld[1] >= -10):
                lpx.append(ld[0])
                lpy.append(ld[1])

        lb = [(-0.0001 * ((np.abs(np.sin(x) * np.sin(y) * np.exp(np.abs(100 - np.sqrt(x**2 + y**2) / np.pi)))) + 1)**0.1) for x, y in zip(lpx, lpy)]    # Calculate the sum of each px[i]+py[i] and store in list b

        lbest = sorted(lb)     # Sort b in lows

        lbx = [lpx[lb.index(val)] for val in lbest]    # Find the corresponding px[i] and py[i]
        lby = [lpy[lb.index(val)] for val in lbest]    #       values for each element in 'best'

        # weights for (lamda_r)^(t+1)
        lw_k = []       # weight vector - contains weights for all I
        ldenom = 0      # denominator for eack w_k_i
        lmu = math.floor(l_ng/2)
        for i in range(1,l_ng+1):
            ldenom = ldenom + (np.log((l_ng+0.5)) - np.log(i))

        for i in range(1, lmu + 1):
            lw_k_i = (np.log((l_ng + 0.5)) - np.log(i)) / ldenom
            lw_k.append(lw_k_i)
        for i in range(lmu+1,l_ng+1):
            lw_k_i = 0
            lw_k.append(lw_k_i)

        lmu_eff = 1 / np.sum(np.square(lw_k))

        #finding phi^(-1)
        al = 0.375
        prob_pg = []
        prob_ng = []

        inval_pg = []
        normval_pg = []

        for i in range(len(bx)):
            valp = (i+1-al)/(len(bx)+1-2*al)
            inval_pg.append(valp)
            phipg = norm.ppf(valp)
            normval_pg.append(phipg)

            mxppg = w_k[i]*(mpg[0] + sigpg*(phipg))
            myppg = w_k[i]*(mpg[1] + sigpg*(phipg))
            exp_pg = math.sqrt((mxppg**2 + myppg**2))
            prob_pg.append(exp_pg)


#             mxppg = w_k[i]*(mpg[0] + sigpg*(norm.ppf((i+1)/(len(bx)+1))))
#             myppg = w_k[i]*(mpg[1] + sigpg*(norm.ppf((i+1)/(len(bx)+1))))
#             exp_pg = math.sqrt((mxppg**2 + myppg**2))
#             prob_pg.append(exp_pg)
            #exp_pg = mxppg

        print("\n")
        print("At generation",k,"P(x) value for L_g =")
        print(inval_pg)
        print("At generation",k,"phi(-1) value for L_g =")
        print(normval_pg)
        print("\n")


        inval_ng = []
        normval_ng = []

        for i in range(len(lbx)):
            valn = (i+1-al)/(len(lbx)+1-2*al)
            inval_ng.append(valn)
            phing = norm.ppf(valn)
            normval_ng.append(phing)

            mxpng = lw_k[i]*(m[0] + sig*(phing))
            mypng = lw_k[i]*(m[1] + sig*(phing))
            exp_ng = math.sqrt((mxpng**2 + mypng**2))
            prob_ng.append(exp_ng)

#             mxpng = lw_k[i]*(m[0] + sig*(norm.ppf((i+1)/(len(lbx)+1))))
#             mypng = lw_k[i]*(m[1] + sig*(norm.ppf((i+1)/(len(lbx)+1))))
#             exp_ng = math.sqrt((mxpng**2 + mypng**2))
#             #exp_ng = mxpng
#             prob_ng.append(exp_ng)
        print("\n")
        print("At generation",k,"P(x) value for L_g+1 =")
        print(inval_ng)
        print("At generation",k,"phi(-1) value for L_g+1 =")
        print(normval_ng)
        print("\n")


        print("phi^(-1)_pg = ",phipg)
        print("phi^(-1)_ng = ",phing)

        print("\n")

        print("weighted expected values of NOS")
        print("\n")

        print("E[l_r^(g)] = ",prob_pg)
        print("E[l_r^(g+1)] = ",prob_ng)

        print("sum{E[l_r^(g)]} = ",sum(prob_pg))
        print("sum{E[l_r^(g+1)]} = ",sum(prob_ng))

        print("\n")

        cpg = (-1)*sum(prob_pg)
        cng = (-1)*sum(prob_ng)
        sigfunc_pg = (cpg*n*mu_eff)/(n - 1 + (cpg**2)*mu_eff)
        sigfunc_ng = (cng*n*lmu_eff)/(n - 1 + (cng**2)*lmu_eff)

        print("lr(g+1) = ",sigfunc_ng)
        print("lr(g) = ",sigfunc_pg)
        print("lr(g+1)/lr(g) = ",sigfunc_ng/sigfunc_pg)
        print("\n")
        print("st-s b4 correction = ",sig)
        # step size correction

        st_sng.append(sigfunc_ng)
        st_spg.append(sigfunc_pg)
        st_div.append(sigfunc_ng/sigfunc_pg)

        popchange.append(abs(l_ng - l_pg))


#         if reall - l < 10:
#             if norm_psig >= chi:
#                 sig = sig
#                 print("step-size correction not done as CSA request step-size increase")
#             else:
#                 sig = sig*(0.5*(sigfunc_ng/sigfunc_pg))
#                 print("scaled step-size correction (pop-dif < 10)")

#         else:
#             if norm_psig >= chi:
#                 sig = sig
#                 print("step-size correction not done as CSA request step-size increase")
#             else:
#                 sig = sig*((sigfunc_ng/sigfunc_pg))
#                 print("unscaled step-size correction")


#         print("\n")
#         print("Trying to figure out why this formulation worked")
#         print("reall = ",reall)
#         print("l = ",l)
#         print("If ",reall,"-",l,"< 10 --> sigma was scaled (if psig okay)")
#         print("\n")

        sig = sig*((sigfunc_ng/sigfunc_pg))





#         if (sum(sum(D)))<10**(-2):
#             sig = sig*(0.5*(sigfunc_ng/sigfunc_pg))
#         else:
#             sig = sig*((sigfunc_ng/sigfunc_pg))



#         sig = sig

        print("Step size correction = sig*[lr(g+1)/lr(g)]")
        print("st-x af correction = ",sig)

        sigTot.append(sig)



        f = (-0.0001 * ((np.abs(np.sin(mx) * np.sin(my) * np.exp(np.abs(100 - np.sqrt(mx**2 + my**2) / np.pi)))) + 1)**0.1)  # function value - to plot

        F.append(f)
        Mean.append(m)
        Cov.append(c)

        print("\n")
        print("Covariance matrix for ",k+1)
        print(c)


#         # Create a grid of x and y values
        x = np.linspace(-100, 100, 500)
        y = np.linspace(-100, 100, 500)
        X, Y = np.meshgrid(x, y)

        Z = np.exp(-(np.square(X - m[0]) * c[0, 0] +
                 (X - m[0]) * (Y - m[1]) * c[0, 1] +
                 np.square(Y - m[1]) * c[1, 1])) / (2 * np.pi * np.sqrt(np.linalg.det(c)))

        # Plot the 2D Gaussian distribution
        plt.figure(3*k+1)
        plt.figure(figsize=(8, 6))
        plt.contourf(X, Y, Z, levels=10, cmap='Reds')
        plt.colorbar()
        plt.scatter(px, py, color='cyan', alpha=0.4, label='xk({})'.format(k+1))
        plt.scatter(m[0], m[1], color='yellow', label='Mean at {}th generation'.format(k+1))
        plt.xlabel('X',fontsize=14)
        plt.ylabel('Y',fontsize=14)
        plt.tick_params(axis='both', which='major', labelsize=16)
        plt.title('2D Gaussian Distribution for generation {}'.format(k+1),fontsize=16)
        plt.grid(True)
        plt.show()



#         # Generate random points from multivariate normal distribution
#         num_points = len(px)
#         points = np.random.multivariate_normal(m, c, num_points)
#         px, py = points[:, 0], points[:, 1]

#         # Create a multivariate normal distribution object
#         mv_normal = multivariate_normal(mean=m, cov=c)

#         # Define confidence level
#         confidence_level = 0.95

#         # Degrees of freedom for the chi-squared distribution
#         df = len(m)

#         # Calculate critical value for the confidence interval
#         critical_value = chi2.ppf(confidence_level, df)

#         # Plot points
#         plt.scatter(px, py, color='blue', label='Generated Points')

#         # Plot contours
#         x, y = np.mgrid[-100:100:.01, -100:100:.01]
#         pos = np.dstack((x, y))
#         contour_levels = mv_normal.pdf(pos)
#         plt.contour(x, y, contour_levels, colors='blue',alpha=0.3)

#         # Plot critical value contour
#         plt.contour(x, y, mv_normal.pdf(pos), levels=[mv_normal.pdf(critical_value)], colors='green')

#         # Identify points inside and outside the confidence interval
#         inside_points = points[np.where(mv_normal.pdf(points) >= mv_normal.pdf(critical_value))]
#         outside_points = points[np.where(mv_normal.pdf(points) < mv_normal.pdf(critical_value))]

#         # Highlight points
#         plt.scatter(inside_points[:, 0], inside_points[:, 1], color='green', label='Inside Confidence Interval')
#         plt.scatter(outside_points[:, 0], outside_points[:, 1], color='red', label='Outside Confidence Interval')

#         # Add legend
#         plt.legend()

#         # Show plot
#         plt.xlabel('X')
#         plt.ylabel('Y')
#         plt.title('Points from Multivariate Normal Distribution with Confidence Intervals')
#         plt.grid(True)
#         plt.show()




        print("\n")
        print("\n")
        print("\n")
        print("\n")

        if abs(F[k] - 0.0)<10**(-2):
            break

        k = k+1



    end_time = time.process_time()

    print("\n")

    cpu_time = end_time - start_time
    print("CPU time for the loop:", cpu_time)

    print("run ",p," : fun_vals = ",F)
    p = p+1



In [ ]:
print("Total sigma variation")
print(sigTot)


plt.figure(1)
plt.plot(sigTot, marker='o', linestyle='-',label='total')
plt.plot(sigCSA, marker='o', linestyle='-',label='CSA')
# plt.plot(range(len(sigTot)), sigTot, marker='o', linestyle='-')
plt.title('Total step-size variation', fontsize=16)
plt.xlabel('Generation',fontsize=16)
plt.ylabel('sigma',fontsize=16)
plt.grid(True)
plt.legend()
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

plt.show()

print("\n")
print("Sigma variation of CSA (both happen)")
print(sigCSA)

plt.figure(2)
plt.plot(F, marker='o', linestyle='-')
plt.title('Function Value',fontsize=16)
plt.xlabel('Generation',fontsize=16)
plt.ylabel('f(E[xi])',fontsize=16)
plt.grid(True)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

print("\n")


plt.figure(3)
plt.plot(real_l, marker='o', linestyle='-', label='real L')
# plt.plot(fixed_l, marker='o', linestyle='-',label = 'fixed L')
# plt.plot(range(len(sigTot)), sigTot, marker='o', linestyle='-')
plt.title('Population size variation', fontsize=16)
plt.xlabel('Generation',fontsize=16)
plt.ylabel('lambda',fontsize=16)
plt.legend()
plt.grid(True)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()

plt.show()



print("\n")
print("Mean", Mean)
Mean = np.array(Mean)
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(Mean[:, 0], Mean[:, 1], color='blue', label='Mean')
ax.plot(Mean[:, 0], Mean[:, 1], color='red')
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

# set axis labels and title
plt.xlabel('Mean[x] = x1',fontsize=16)
plt.ylabel('Mean[y] = x2',fontsize=16)
plt.title('Plot of mean at each generation',fontsize=16)
plt.legend()
plt.grid()

# show plot
plt.tick_params(axis='both', which='major', labelsize=16)
plt.show()